# Quantum Query Algorithms — Detailed Notes (Session 4)
**Course:** CS490/5590 — Quantum Computing Applications in Data Science, AI, & Deep Learning  
**Instructor:** Luke Miller  
**Level:** Graduate / Advanced Undergraduate  

> **Purpose of these notes.** A self-contained, expanded companion to the Session-4 slides. We revisit the query (oracle) model, walk through Deutsch–Jozsa (DJ) and Bernstein–Vazirani (BV) in detail, supply Qiskit snippets, give complexity proofs, and suggest AI/DS connections. Mini-exercises with short solutions are at the end.

---

## Session Overview  

1. Quick recap (entanglement, Bell resource)  
2. Query (oracle) model — classical vs quantum  
3. Black-box oracles (bit-flip vs phase style)  
4. Deutsch–Jozsa algorithm  
5. Bernstein–Vazirani algorithm  
6. Query-complexity analysis  
7. Applications to AI / data science  
8. Q&A  

---

## 0) Recap from Session 3  

| Topic | Take-away |
|-------|-----------|
| **Entanglement** | Non-separable, enables stronger-than-classical correlations |
| **Bell states** | Prepared via $H$+CNOT; used in teleportation, super-dense coding |
| **Teleportation** | Moves unknown state using 2 cbits + Bell pair |
| **Non-locality** | CHSH violation confirms absence of local hidden variables |

**Bridge to today.**  Query algorithms exploit *superposition + interference* (often plus entanglement) to interrogate a black-box function more efficiently than any classical strategy.

---

## 1) The query (oracle) model  

### 1.1 Classical
- Oracle = subroutine returning $f(x)$ on input $x$.  
- Complexity measured in **number of oracle calls** (time ignoring local computation).

### 1.2 Quantum
- Oracle implemented as a *unitary* $O_f$. Two common encodings:  
  1. **Bit-flip oracle** (standard in DJ/BV):  
      $ O_f:|x\rangle|b\rangle \mapsto|x\rangle\,|b\oplus f(x)\rangle.$
  2. **Phase oracle** (used in Grover):  
     $O_f^{\phi}:|x\rangle \mapsto(-1)^{f(x)}|x\rangle.$
- Superposition query: prepare $\tfrac1{\sqrt{2^n}}\sum_x|x\rangle$ ⇒ single call “evaluates” $f$ on *all* $x$ coherently.

> **Why it matters.**  Many quantum speedups arise solely from reduced query complexity.

### 1.3 Formal set-up
For Boolean $f:\{0,1\}^n\to\{0,1\}$ the bit-flip oracle is reversible because the target qubit stores $b\oplus f(x)$. Controlled phase versions can be built with one extra Hadamard.

---

## 2) Black-box (oracle) construction notes  

- **Unitary requirement.** Add extra work-bits / clean ancillas so overall transformation is bijective.  
- **Parallelism.** Superposition allows querying *every* $x$ at once, but measurement later reveals at most $n$ classical bits — interference is used to compress the global property we care about.  
- **Why classical cannot**: Classical calls interrogate *one* $x$ per call; no interference across branches.

---

## 3) Deutsch–Jozsa (DJ) problem  

**Promise.** Oracle $f:\{0,1\}^n\to\{0,1\}$ is either *constant* (all outputs equal) or *balanced* (outputs 0 on exactly half of inputs). Decide which.

| Approach | # Queries (worst-case) |
|----------|------------------------|
| Classical deterministic | $2^{\,n-1}+1$ |
| Classical randomized (prob ≤ ½) | $O(1)$ |
| **Quantum deterministic** | **1** |

---

### 3.1 DJ algorithm  

1. **Init** $|0\rangle^{\!\otimes n}\;|1\rangle$.  
2. Apply $H^{\otimes (n+1)}$. Auxiliary qubit becomes $|-\rangle$; data register in uniform superposition.  
3. **Oracle call** $O_f$.  Acts as phase kickback:  
   $|x\rangle|-\rangle \mapsto (-1)^{f(x)}|x\rangle|-\rangle$.  
   (Proof: bit-flip on $|-\rangle$ = phase flip.)  
4. Apply $H^{\otimes n}$ to the first $n$ qubits.  
5. **Measure** the first $n$ qubits.  
   - Outcome all-zero ⇒ constant.  
   - Anything else ⇒ balanced.

**Why it works.**  
Amplitude for $|0\rangle^{\!\otimes n}$ after step 4 is  
$
\frac{1}{2^n}\sum_{x\in\{0,1\}^n}(-1)^{f(x)}.
$
- Constant $f$: sum = $\pm 2^n$ ⇒ amplitude ±1.  
- Balanced $f$: equally many +1 and –1 ⇒ amplitude 0.

> **Mini-fact.** DJ is the first *exponential* gap in deterministic query complexity (though with an artificial promise).

---

#### 3.2 Qiskit template
```python
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator

def dj_oracle_balanced(n):
    """Example balanced oracle: f(x)=x0 XOR ... XOR xn-1."""
    qc = QuantumCircuit(n+1)
    for i in range(n):
        qc.cx(i, n)         # copy each bit into target (XOR)
    return qc

def dj_algorithm(n, oracle):
    qc = QuantumCircuit(n+1, n)
    qc.x(n); qc.h(range(n+1))
    qc.compose(oracle, inplace=True)
    qc.h(range(n))
    qc.measure(range(n), range(n))
    return qc

n = 3
oracle = dj_oracle_balanced(n)
qc = dj_algorithm(n, oracle)
print(qc.draw('text'))
backend = AerSimulator()
print(backend.run(transpile(qc, backend), shots=1024).result().get_counts())
```  

---

## 4) Bernstein–Vazirani (BV) problem  

**Oracle.** $f_s(x)=s\cdot x\;(\text{mod }2)$ where $s\in\{0,1\}^n$ is a hidden string and “$\cdot$” is bitwise inner product (parity). Task: output $s$.

| Approach | # Queries | Success prob |
|----------|-----------|--------------|
| Classical deterministic | $n$ | 1 |
| Classical randomised | $n$ | 1 |
| **Quantum deterministic** | **1** | 1 |

### 4.1 BV algorithm  

1. Init $|0\rangle^{\!\otimes n}\;|1\rangle$.  
2. $H^{\otimes (n+1)}$.  
3. Oracle $O_f$ (bit-flip) → phase kickback $(-1)^{s\cdot x}|x\rangle$.  
4. $H^{\otimes n}$ on data qubits.  
5. Measure first $n$ qubits — result is $|s\rangle$.

**Fourier viewpoint.** Hadamard = quantum Fourier transform over $\mathbb{Z}_2^n$; inner product in exponent becomes basis index after second Hadamard.

---

#### 4.2 Qiskit snippet
```python
def bv_oracle(s):
    n = len(s)
    qc = QuantumCircuit(n+1)
    for i, bit in enumerate(s[::-1]):      # reverse for Qiskit little-endian
        if bit == '1':
            qc.cx(i, n)
    return qc

s = '10110'
oracle = bv_oracle(s)
qc = dj_algorithm(len(s), oracle)          # same wrapper
backend = AerSimulator()
counts = backend.run(transpile(qc, backend), shots=1).result().get_counts()
print(counts)  # should output {'10110': 1}
```

---

## 5) Query-complexity analysis tools

| Method | Idea | Typical use |
|--------|------|-------------|
| **Decision-tree** | Each classical query splits cases; depth = # queries. | Lower bound classical |
| **Adversary** | Spectral bound on distinguishing inputs with few queries. | Quantum lower bounds |
| **Polynomial** | Represent acceptance prob as low-deg polynomial. | Tight bounds for DJ, PARITY |

### Results recap
- **DJ:** 1 quantum vs $\Omega(2^{\,n})$ classical deterministic.  
- **BV:** 1 vs $\Omega(n)$.  
- **PARITY:** 1 quantum vs $n$ classical — same trick, phase oracle.

---

## 6) Applications & relevance to AI / DS  

| Area | Possible use of oracle style |
|------|------------------------------|
| **Feature testing** | Oracle = “does sample $x$ have property $P$?” Superposition tests all $x$. |
| **Hidden shift / pattern** | BV is a toy for hidden-string learning; generalisations (e.g., Simon, HSP) relate to cryptanalysis. |
| **Optimization (QAOA)** | Phase oracles encode cost; number of oracle layers ↔ problem depth. |
| **Quantum kernels** | Phase oracles implement kernel phases $e^{i\phi(x,x')}$. |

> **Realism caution.** In practice the oracle must be compiled from gates; circuit depth may outweigh theoretical query savings on NISQ devices.

---

## 7) Worked examples & proofs  

**Example A — DJ amplitude derivation.** Calculate amplitude for $|0\rangle^{\!\otimes n}$ given balanced $f$. Use pair-cancellation to show zero.  

**Example B — BV algebra (n=3).** Suppose $s=110$. Show the sign pattern after oracle is $(-1)^{x_1\oplus x_2} |x\rangle$. Apply $H^{\otimes3}$ to recover $|s\rangle$.  

**Example C — Classical lower bound for DJ.** Any deterministic algo must query > $2^{n-1}$ inputs else an adversary chooses remaining half to fool it.

---

## 8) Mini-exercises (solutions in Appendix)

1. Implement a *constant* DJ oracle in Qiskit and verify the circuit outputs all-zero.  
2. Prove that BV’s oracle induces a diagonal matrix with phases $(-1)^{s\cdot x}$.  
3. Design a phase oracle for PARITY $f(x)=x_0\oplus\cdots\oplus x_{n-1}$. How many classical deterministic queries are required?  
4. Show how to modify DJ to use a **phase oracle** instead of the bit-flip variant (no ancilla qubit).  
5. Suppose noise flips the oracle output with probability $p$. What’s the failure probability of BV? (Assume single run.)  

---

## 9) FAQ / common pitfalls  

- **“Where does the minus sign come from in phase kickback?”** Auxiliary qubit in $|-\rangle=(|0\rangle-|1\rangle)/\sqrt2$ converts $b\oplus f(x)$ into a phase $(-1)^{f(x)}$.  
- **“Why can’t classical randomisation match DJ?”** With randomness you can reduce *expected* queries but still need $\Omega(1)$ probability of error; DJ is *deterministic* with one query.  
- **“Aren’t real oracles huge circuits?”** Yes; query models idealise cost. Practical advantage depends on compiling $f$ efficiently.  
- **“BV speedup is only linear.”** True, but deterministic and foundational for hidden-shift family; still showcases the *parallel* Fourier trick.

---

## 10) Summary (Session 4)

- **Oracle / query model** measures complexity by number of black-box calls.  
- Quantum algorithms exploit *superposition* + *interference*: query once, gather global property.  
- **Deutsch–Jozsa:** 1 query vs $2^{n}$ deterministic classical.  
- **Bernstein–Vazirani:** 1 query vs $n$ classical; extracts hidden string.  
- These ideas generalise to Simon’s, Shor’s, and underpin quantum speedups in search (next session: Grover).  
- AI/DS: conceptually map to property testing, pattern discovery, kernel phases — but oracle realisation cost is the bottleneck.

---

## 11) Looking ahead  

- **Next Session:** Unstructured search & amplitude amplification (Grover).  
- **Homework reminder:** HW-1 covers Sessions 1-4. Implement DJ & BV in Qiskit, answer mini-exercise questions, and reflect on oracle realism.

---

## 12) Appendix — solutions to mini-exercises (sketch)

1. Constant-zero oracle = identity; constant-one oracle = X on ancilla before and after queries. Circuit outputs `0…0`.  
2. Bit-flip on $|-\rangle$ ⇒ $(-1)^{f(x)}$. Since $f_s(x)=s\cdot x$ is linear, phase is $(-1)^{s\cdot x}$.  
3. PARITY needs $n$ deterministic classical queries; quantum phase oracle solves with 1 via same BV circuit.  
4. Replace ancilla with global phase: implement oracle $U_f|x\rangle = (-1)^{f(x)}|x\rangle$, omit $|1\rangle$ qubit, rest identical.  
5. If oracle error flips phase with prob $p$, measurement yields wrong $s$ with probability $p$ (because wrong phase on any $x$ breaks perfect interference).

